# Alumno: Gerardo de Miguel González

### Carga de dataset y visualización

#### Caso Lineal

In [1]:
#::GMG::Cargo los datos del problema lineal (nube de puntos ejemplo)
lin <- read.csv('data/lineal.csv',header = F)

In [2]:
str(lin)

'data.frame':	15 obs. of  3 variables:
 $ V1: num  0.272 0.506 0.526 0.932 0.369 0.524 0.75 0.005 0.641 0.827 ...
 $ V2: num  0.987 0.371 0.9 0.968 0.938 0.196 0.594 0.972 0.926 0.617 ...
 $ V3: int  0 1 0 1 0 1 1 0 0 1 ...


In [3]:
#::GMG::Obtengo los índices de los puntos de una de las clases
ind_l <- which(lin[,3]==0)

In [4]:
#::GMG::Dibujo los puntos con las dos clases
#::nota::
# https://support.rstudio.com/hc/en-us/community/posts/213106048-Figure-size-in-R-Notebook-document
# https://www.rdocumentation.org/packages/graphics/versions/3.5.3/topics/plot
# https://www.rdocumentation.org/packages/graphics/versions/3.5.2/topics/lines
par(mar = c(2,5,1,1))
plot(lin[ind_l,1],lin[ind_l,2],
     type='p',
     xlim=c(0,1),ylim=c(0,1),
     xlab = 'x', ylab = 'y',
     col='blue')
lines(lin[-ind_l,1],lin[-ind_l,2],
      type='p',
      col='red')

### Single-layer perceptron (slp)

**::GMG::** S8-S10, S18-S22 (S01-Intro-FeedForward.pdf), S2-S13 (S03-Lab-Backprop.pdf)

In [5]:
#::GMG::Elijo la función de activación que voy a aplicar (S8-S10)
#       a las neuronas (logistic)
#::nota:: f'(x) = f(x) (1 - f(x)) (S10, S01-Intro-FeedForward.pdf)
sigmoid <- function(z) {
    1/(1 + exp(-z))
}

In [6]:
#::GMG::Construyo la función de FF de la NN (S8, S01-Intro-FeedForward.pdf)
feedforward <- function(x, w) {
  #::nota:: x %*% t(w) resulta en vector columna 
  #         (si s hace w %*% t(x) da lo mismo PERO sale un vector fila) 
  return (sigmoid(x %*% t(w)))
}

In [7]:
#::GMG::Construyo la función de aprendizaje backprop
#       con gradient descent (S7-10, S03-Lab-Backprop.pdf)
backprop <- function(y,x, epochs = 10, eta = 0.1) {
    #::GMG::vector columna de salida out x 1
    b <- as.matrix(y)
    #::GMG::vector columna de entrada in + 1 x 1
    #::nota::se añade el bias
    # https://stackoverflow.com/questions/2480650/role-of-bias-in-neural-networks
    a <- cbind (1, as.matrix(x))
    #::GMG::número de pesos a generar (a input neurons, b output neurons)
    n <- c(ncol(a),ncol(b))
    #::GMG::Generación aleatoria de pesos iniciales out x in+1
    w <- matrix(data = runif(prod(n), 
                min = -1, max = 1),
                nrow = n[2], ncol = n[1]
               )
    #::GMG::Ajuste de pesos con gradient descent (no reg, no inercia)
    for (j in 1:epochs) {
        #::GMG::Primero se "propaga hacia delante"
        b_hat <- feedforward(a,w)
        #::GMG::Luego se actualizan los "pesos" con learning rate "eta"
        error <- b - b_hat
        #::GMG:: (b - b_hat) * f'(B)
        aux <- error * b_hat * (1 - b_hat)
        delta_w <- eta * (t(aux) %*% a)
        w <- w + delta_w
        #::GMG::se calcula la suma del error cuadrático a minimizar 
        sse <- (t(error) %*% error)/2
    }
    #::GMG::Quito el nombre de las columnas de la matriz de pesos (?)
    #colnames (w) <- NULL
    return(list(error = sse, pesos = w))
}

### Prueba con dataset

In [8]:
set.seed(1)

In [9]:
modelo_bp <- backprop(y = lin[,3], x = lin[,-3], epochs = 500, eta = 0.01)

In [10]:
modelo_bp$error

1.166098


In [11]:
modelo_bp$pesos

,V1,V2
-0.06174339,1.484874,-0.9486623


In [12]:
#::GMG::La línea de separación es la proyección en xy de la curva
#       de salida
linea <- function(w, x, Z) {
    return((Z - w[1] - w[2]*x)/w[3])
}

In [13]:
#::GMG::Vuelvo a dibujar el dataset con la línea de separación
par(mar = c(2,5,1,1))
plot(lin[ind_l,1],lin[ind_l,2],
     type='p',
     xlim=c(0,1),ylim=c(0,1),
     xlab = 'x', ylab = 'y',
     col='blue')
lines(lin[-ind_l,1],lin[-ind_l,2],
      type='p',
      col='red')
lines(c(0,1), c(linea(modelo_bp$pesos, 0, 0), 
      linea(modelo_bp$pesos, 1, 0)), 
      col='darkgreen', lwd=1.4)

### Comparación con biblioteca neuralnet

In [14]:
# https://cran.r-project.org/web/packages/neuralnet/index.html
# https://www.rdocumentation.org/packages/neuralnet/versions/1.44.2
# https://datascienceplus.com/neuralnet-train-and-test-neural-networks-using-r/
# https://www.datacamp.com/community/tutorials/neural-network-models-r
library('neuralnet')

In [15]:
set.seed(1)

In [16]:
#::GMG::Recuerdo cómo es el dataset
str(lin)

'data.frame':	15 obs. of  3 variables:
 $ V1: num  0.272 0.506 0.526 0.932 0.369 0.524 0.75 0.005 0.641 0.827 ...
 $ V2: num  0.987 0.371 0.9 0.968 0.938 0.196 0.594 0.972 0.926 0.617 ...
 $ V3: int  0 1 0 1 0 1 1 0 0 1 ...


In [17]:
#::GMG::Pongo los parámetros iguales al caso 'slp'
#::nota::no hidden layers -> slp (!)
# https://www.rdocumentation.org/packages/neuralnet/versions/1.44.2/topics/neuralnet
modelo_nn <- neuralnet(formula = V3 ~ V2 + V1, 
                       data = lin, 
                       hidden = 0,
                       learningrate = 0.01,
                       algorithm = 'backprop',
                       err.fct = 'sse',
                       act.fct = 'logistic',
                       linear.output = FALSE
                      )

In [18]:
modelo_nn$result.matrix

,1
error,0.062918567575
reached.threshold,0.009999958349
steps,34007.000000000000
Intercept.to.V3,-0.839224993116
V2.to.V3,-8.621918354500
V1.to.V3,11.796358446172


In [19]:
#::GMG::Los pesos se devuelven en una lista
modelo_nn$weights

-0.8392249931
-8.6219183545
11.7963584462


In [20]:
#::GMG::El orden de los pesos en neuralnet es 
#       intercept, V2, V1 
#       y yo supongo en linea() un orden
#       intercept, V1, V2
w <- unlist(modelo_nn$weights)[c(1,3,2)]
print (w)

[1] -0.8392249931 11.7963584462 -8.6219183545


In [21]:
#::GMG::Veo la línea de separación que resulta
par(mar = c(2,5,1,1))
plot(lin[ind_l,1],lin[ind_l,2],
     type='p',
     xlim=c(0,1),ylim=c(0,1),
     xlab = 'x', ylab = 'y',
     col='blue')
lines(lin[-ind_l,1],lin[-ind_l,2],
      type='p',
      col='red')
lines(c(0,1), 
      c(linea(w, 0, 0), 
        linea(w, 1, 0)
       ), 
      col='darkgreen', lwd=1.4)

### Notas de clase

In [21]:
lin[,3]

[1] 0 1 0 1 0 1 1 0 0 1 1 1 0 0 1

In [12]:
#::GMG::Obtengo el vector columna (matriz) de entrada de lo que tengo
a_l <- as.matrix(lin[,-3])
dim(a_l)

[1] 15  2

In [13]:
#::GMG::Incluyo el bias 
a_l <- cbind(a_l,rep(1,nrow(a_l)))
dim(a_l)

[1] 15  3

In [15]:
a_l

V1,V2,
0.272,0.987,1
0.506,0.371,1
0.526,0.900,1
0.932,0.968,1
0.369,0.938,1
0.524,0.196,1
0.750,0.594,1
0.005,0.972,1
0.641,0.926,1
0.827,0.617,1


In [16]:
a_l_2 <- cbind (as.matrix(lin[,-3]), 1)
dim (a_l_2)

[1] 15  3

In [17]:
a_l_2

V1,V2,
0.272,0.987,1
0.506,0.371,1
0.526,0.900,1
0.932,0.968,1
0.369,0.938,1
0.524,0.196,1
0.750,0.594,1
0.005,0.972,1
0.641,0.926,1
0.827,0.617,1


In [18]:
#::GMG::Calculo el vector columna (matriz) de salida de lo que quiero
b_l <- as.matrix(lin[,3])
dim(b_l)

[1] 15  1

In [19]:
#::GMG::Construyo las neuronas del single-layer perceptron
#::nota::tres neuronas de entrada y 1 una neurona de salida 
#        sin capas (layers) ocultas
neurons <- c(ncol(a_l),ncol(b_l))
neurons

[1] 3 1

In [ ]:
#::GMG::Recordatorio para hacer el producto escalar de la función sigmoid
# Producto escalar c(1,2,3)%*%c(3,2,1)
#t(b)%*%b
#8

In [20]:
#::GMG::Calculo una matriz de pesos inicial de forma aleatoria
#::nota::uso la función vectorizada prod()
# https://www.rdocumentation.org/packages/base/versions/3.5.2/topics/prod
W <- matrix(data = runif(prod(neurons), min = 0, max = 1),
            nrow = neurons[2], ncol = neurons[1])
dim(W)

[1] 1 3

In [21]:
W

0.2655087,0.3721239,0.5728534


In [24]:
bout <- sigmoid(a_l %*% t(W))

In [27]:
dim(a_l) 
dim(t(W))

[1] 15  3

[1] 3 1

In [31]:
a_l %*% t(W) - t(W %*% t(a_l))

0.000000e+00
0.000000e+00
0.000000e+00
0.000000e+00
2.220446e-16
0.000000e+00
0.000000e+00
0.000000e+00
0.000000e+00
0.000000e+00
-1.110223e-16


In [34]:
dim(bout)
dim (b_l)

[1] 15  1

[1] 15  1

In [36]:
error <- b_l-bout
aux<- error*bout*(1-bout)
Wdelta <- t(aux)%*%a_l

In [37]:
error

-0.7334814
0.3004284
-0.7402796
0.2349581
-0.7349489
0.3132613
0.2703147
-0.7182678
-0.7479354
0.2646323
0.3044511


In [46]:
dim(error)
dim(t(error))

[1] 15  1

[1]  1 15

In [38]:
dim(aux)

[1] 15  1

In [39]:
dim (a_l)

[1] 15  3

In [40]:
dim(Wdelta)

[1] 1 3

In [41]:
dim(W)

[1] 1 3

In [42]:
0.1 * Wdelta

V1,V2,
-0.0009174253,-0.06310155,-0.05555552


In [43]:
Wdelta

V1,V2,
-0.009174253,-0.6310155,-0.5555552


In [45]:
print(a_l)

         V1    V2  
 [1,] 0.272 0.987 1
 [2,] 0.506 0.371 1
 [3,] 0.526 0.900 1
 [4,] 0.932 0.968 1
 [5,] 0.369 0.938 1
 [6,] 0.524 0.196 1
 [7,] 0.750 0.594 1
 [8,] 0.005 0.972 1
 [9,] 0.641 0.926 1
[10,] 0.827 0.617 1
[11,] 0.629 0.232 1
[12,] 0.818 0.295 1
[13,] 0.112 0.318 1
[14,] 0.351 0.813 1
[15,] 0.739 0.706 1
